In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/adapter_model.safetensors
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/adapter_config.json
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/README.md
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/tokenizer.json
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/tokenizer_config.json
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/chat_template.jinja
/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/special_tokens_map.json


# This file merges the adapter created by fine tunnig the llama3 8B instruct model using PERT method and the base llama3 8B instruct model.

In [2]:
!pip install -U peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.7 MB/s eta 0:00:00:00:0100:01
  Attempting un

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import shutil
# from google.colab import files


2025-07-30 17:03:00.830100: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753894981.039422      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753894981.096297      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())          # should print 2
print(torch.cuda.get_device_name(0))      # e.g., 'Tesla T4'
print(torch.cuda.get_device_name(1))      # e.g., 'Tesla T4'


True
2
Tesla T4
Tesla T4


In [5]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token= user_secrets.get_secret("HF_TOKEN")

login(token= token)


In [6]:
shutil.copytree("/kaggle/input/stufit-rag-chatbot-lora-adapter-final/lora_adapter_latest/","/kaggle/working/lora_adapter_latest/")

'/kaggle/working/lora_adapter_latest/'

In [7]:
# Define paths
adapter_path = "/kaggle/working/lora_adapter_latest/"
output_path = "/kaggle/working/llama3-merged-model"

In [8]:
# Load PEFT config to retrieve base model name
peft_config = PeftConfig.from_pretrained(adapter_path)

In [9]:
# Load base model on CPU (merging MUST be done in full precision)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16,
    load_in_4bit=False,
    load_in_8bit=False
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [10]:
# Load LoRA adapter into base model
model = PeftModel.from_pretrained(base_model, adapter_path)

In [11]:
# Merge and unload LoRA weights
model = model.merge_and_unload()

In [12]:
# Save the merged model & tokenizer
model.save_pretrained(output_path)
tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)
tokenizer.save_pretrained(output_path)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

('/kaggle/working/llama3-merged-model/tokenizer_config.json',
 '/kaggle/working/llama3-merged-model/special_tokens_map.json',
 '/kaggle/working/llama3-merged-model/chat_template.jinja',
 '/kaggle/working/llama3-merged-model/tokenizer.json')

In [18]:
import shutil
from IPython.display import FileLink

# import shutil

# # Replace with your actual folder name
# folder_to_zip = '/kaggle/working/llama3-merged-model'
# temp_zip_path = '/kaggle/tmp/llama3-merged-model.zip'
# shutil.make_archive('/kaggle/tmp/llama3-merged-model', 'zip', folder_to_zip)



# # Move zip to working directory (downloadable)
# shutil.move('/kaggle/tmp/my_folder.zip', '/kaggle/working/my_folder.zip')

# Display download link
display(FileLink('/kaggle/working/llama3-merged-model/model-00001-of-00004.safetensors'))
display(FileLink('/kaggle/working/llama3-merged-model/model-00002-of-00004.safetensors'))
display(FileLink('/kaggle/working/llama3-merged-model/model-00003-of-00004.safetensors'))
display(FileLink('/kaggle/working/llama3-merged-model/model-00004-of-00004.safetensors'))


/kaggle/working/llama3-merged-model/model-00001-of-00004.safetensors

/kaggle/working/llama3-merged-model/model-00002-of-00004.safetensors

/kaggle/working/llama3-merged-model/model-00003-of-00004.safetensors

/kaggle/working/llama3-merged-model/model-00004-of-00004.safetensors

In [20]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN2")  # (Save your token in Kaggle "Secrets")
login(token=hf_token)


In [24]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "arindam-117/merged_model_lama_3"  # Replace with your HF username and repo name
folder_path = "/kaggle/working/llama3-merged-model"


# List all files in the model folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        print(f"Uploading {filename} ...")
        api.upload_file(
            path_or_fileobj=file_path,
            path_in_repo=filename,
            repo_id=repo_id,
            repo_type="model",  # Use "dataset" for dataset repos
        )
        print(f"Uploaded {filename}.")

print("All files uploaded one-by-one.")

Uploading model-00003-of-00004.safetensors ...


Uploading...:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Uploaded model-00003-of-00004.safetensors.
Uploading model-00004-of-00004.safetensors ...


Uploading...:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Uploaded model-00004-of-00004.safetensors.
Uploading model.safetensors.index.json ...
Uploaded model.safetensors.index.json.
Uploading tokenizer.json ...


...

Uploaded tokenizer.json.
Uploading config.json ...
Uploaded config.json.
Uploading special_tokens_map.json ...
Uploaded special_tokens_map.json.
Uploading chat_template.jinja ...
Uploaded chat_template.jinja.
Uploading model-00001-of-00004.safetensors ...


Uploading...:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Uploaded model-00001-of-00004.safetensors.
Uploading tokenizer_config.json ...
Uploaded tokenizer_config.json.
Uploading model-00002-of-00004.safetensors ...


Uploading...:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Uploaded model-00002-of-00004.safetensors.
Uploading generation_config.json ...
Uploaded generation_config.json.
All files uploaded one-by-one.
